In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [2]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'march'
new_month = 'april'

In [3]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='assets',index_col='Ticker')
#change call to new filepath
coinmarketcap_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='coinmarketcap',index_col='Ticker')

In [4]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [5]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.loc['VEN'] = df.loc['VET']
df.index = map(lambda x: x.lower(), df.index)

In [72]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Available Supply'] = (d_messari['data']['supply']['liquid'])
    df_all['Supply % Issued'] = df_all['Available Supply'] / (df_all['Y2050 Supply']+0.0001)
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Available Supply']
    df_all['Category'] = (d_messari['data']['misc_data']['categories'][0])
    df_all['Sectors'] = (d_messari['data']['misc_data']['sectors'][0])
    try:
        df_all['Category2'] = (d_messari['data']['misc_data']['categories'][1])
    except:
        pass
    try:
        df_all['Sectors2'] = (d_messari['data']['misc_data']['sectors'][1])
    except:
        pass
    return df_all

In [73]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index:
    try:
        metrics = get_messari_data(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Available Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        df.loc[t,('Category')] = metrics['Category']
        df.loc[t,('Sectors')] = metrics['Sectors']
        try:
            df.loc[t,('Category2')] = metrics['Category2']
        except:
            pass
        try:
            df.loc[t,('Sectors2')] = metrics['Sectors2']
        except:
            pass
    except:
        pass

In [103]:
t = 'IOTA'
metrics = get_messari_data(t)
t = 'IOT'
metrics_log[t] = metrics
df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
df.loc[t,('Available Supply')] = metrics['Available Supply']
df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
df.loc[t,('Price')] = metrics['Price']
df.loc[t,('Marketcap')] = metrics['Marketcap']
df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
df.loc[t,('Category')] = metrics['Category']
df.loc[t,('Sectors')] = metrics['Sectors']
try:
    df.loc[t,('Category2')] = metrics['Category2']
except:
    pass
try:
    df.loc[t,('Sectors2')] = metrics['Sectors2']
except:
    pass

t = 'VET'
metrics = get_messari_data(t)
t = 'VET'
metrics_log[t] = metrics
df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
df.loc[t,('Available Supply')] = metrics['Available Supply']
df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
df.loc[t,('Price')] = metrics['Price']
df.loc[t,('Marketcap')] = metrics['Marketcap']
df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
df.loc[t,('Category')] = metrics['Category']
df.loc[t,('Sectors')] = metrics['Sectors']
try:
    df.loc[t,('Category2')] = metrics['Category2']
except:
    pass
try:
    df.loc[t,('Sectors2')] = metrics['Sectors2']
except:
    pass

In [104]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [105]:
#df.loc['IOT'] = df.loc['IOTA']
#df.loc['VET'] = df.loc['VEN']

In [106]:
#read in other data sources
onchainfx_db = df

In [107]:
#remove duplicates
print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

691
146
145
691
146
145


In [108]:
#fix any ticker mapping issues
onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})

In [110]:
onchainfx_db.loc['VET']

Coin                                 VeChain
Class                                      2
Sector                                   NaN
Quality Exchange?                          1
ERC20                                      1
ICO ok?                                    1
Eligible                                   1
Y2050 Supply                     8.67163e+10
Available Supply                 8.54663e+10
Supply % Issued                     0.985585
Price                             0.00566419
Marketcap                        4.91177e+08
Current Marketcap                4.84097e+08
Category                      Infrastructure
Sectors              Application Development
Sectors2                                 NaN
Category2                                NaN
Name: VET, dtype: object

In [111]:
#onchainfx_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\ocfx_'+new_month+'_2019.xlsx',\
#index_col='Ticker')

In [112]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [113]:
#main function to create table of all assets
def create_universe(onchainfx,coinmarketcap,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    for t in eligibility.index:
        if t in coinmarketcap.index:
            eligibility.loc[t,('Volume')] = coinmarketcap.loc[t,('Volume (30d)')]
        else:
            print (t,' no volume data!')
            eligibility.loc[t,('Volume')] = 1
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [114]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [115]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,coinmarketcap_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

ANT  no volume data!
CENNZ  no volume data!
ECN  no volume data!
EDGE  no volume data!
GBYTE  no volume data!
HC  no volume data!
HC  no volume data!
LBC  no volume data!
MCAP  no volume data!
MLN  no volume data!
NMR  no volume data!
PLR  no volume data!
POT  no volume data!
SPANK  no volume data!
VEN  no volume data!
IOTA  no volume data!
IOTA  no other data!
duplicate tickers


In [116]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Float Pass,Volume,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
BTM,2.386247e+08,1.139148e+08,4.773806e-01,1.002499e+09,0.113631,2.100000e+09,1,84940479.0,0.745649,1,0.0,2.0,0.0,2.0,0
CTXC,4.410859e+07,2.203903e+07,4.996539e-01,1.497925e+08,0.147130,2.997925e+08,1,68819301.0,3.122611,1,0.0,2.0,0.0,2.0,0
DCN,3.512462e+08,1.434938e+07,4.085278e-02,3.268222e+11,0.000044,8.000000e+12,0,4278048.0,0.298135,1,1.0,3.0,1.0,2.0,0
DENT,8.824704e+07,3.896678e+07,4.415648e-01,4.415648e+10,0.000882,1.000000e+11,1,51044895.0,1.309959,1,0.0,3.0,0.0,2.0,0
DRGN,4.230558e+07,2.326777e+07,5.499929e-01,2.384219e+08,0.097591,4.335000e+08,1,5124001.0,0.220219,1,0.0,3.0,0.0,2.0,0
ETN,0.000000e+00,5.497335e+07,9.174404e+13,9.174404e+09,0.005992,0.000000e+00,1,6117669.0,0.111284,1,0.0,3.0,0.0,2.0,0
GRIN,2.411448e+09,1.494235e+07,6.196424e-03,6.051540e+06,2.469182,9.766182e+08,0,304981198.0,20.410520,1,0.0,1.0,0.0,1.0,0
R,1.289235e+08,6.245697e+07,4.844500e-01,4.844500e+08,0.128923,1.000000e+09,1,47805965.0,0.765422,1,0.0,3.0,0.0,2.0,0
RHOC,1.930406e+07,8.378756e+06,4.340412e-01,3.737899e+08,0.022416,8.611852e+08,1,6234254.0,0.744055,1,0.0,3.0,0.0,2.0,0
HT,1.207821e+09,1.207826e+08,1.000004e-01,5.000020e+07,2.415643,5.000000e+08,1,303961237.0,2.516597,1,0.0,3.0,1.0,2.0,0


In [117]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [118]:
universe = universe.rename({'HSR' : 'HC'})
universe

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Float Pass,Volume,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible,Rank
BTC,8.444514e+10,7.087620e+10,0.839316,1.761179e+07,4024.360134,2.098350e+07,1,1.924724e+11,2.715613,1,1.0,1.0,0.0,3.0,1,1
XRP,3.092121e+10,7.081982e+09,0.229033,2.290331e+10,0.309212,1.000000e+11,1,1.638648e+10,2.313827,1,1.0,2.0,0.0,3.0,1,2
ETH,1.724895e+10,1.458264e+10,0.845422,1.048759e+08,139.046625,1.240516e+08,1,9.705392e+10,6.655442,1,1.0,2.0,0.0,3.0,1,3
XLM,1.532711e+10,3.675730e+09,0.239819,3.420799e+10,0.107452,1.426409e+11,1,4.197937e+09,1.142069,1,1.0,1.0,0.0,3.0,1,4
EOS,7.503525e+09,3.950492e+09,0.526485,9.109232e+08,4.336800,1.730198e+09,1,2.927942e+10,7.411589,1,1.0,2.0,0.0,3.0,1,5
LTC,5.177009e+09,3.769780e+09,0.728177,6.108203e+07,61.716675,8.388348e+07,1,3.052390e+10,8.096997,1,1.0,1.0,0.0,3.0,1,6
BCH,3.618740e+09,3.051518e+09,0.843254,1.769437e+07,172.456977,2.098343e+07,1,8.648583e+09,2.834190,1,1.0,1.0,0.0,3.0,1,7
ADA,2.811271e+09,2.025847e+09,0.720616,3.029128e+10,0.066879,4.203526e+10,1,6.126591e+08,0.302421,1,1.0,2.0,0.0,3.0,1,8
TRX,2.629005e+09,1.562660e+09,0.594392,6.671825e+10,0.023422,1.122462e+11,1,6.263859e+09,4.008459,1,1.0,3.0,1.0,3.0,1,9
DASH,1.768856e+09,8.155037e+08,0.461035,8.713552e+06,93.590267,1.890000e+07,1,5.769024e+09,7.074185,1,1.0,1.0,0.0,3.0,1,10


In [119]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained TRX
Retained BSV
10 full! Do not add  DASH
10 full! Do not add  BNB


In [120]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained DCR
Retained ZIL
Retained LSK
Retained ZRX
20 full! Do not add  BAT
20 full! Do not add  STEEM
20 full! Do not add  HOT
20 full! Do not add  QTUM


In [121]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained XVG
Retained POWR
Retained STRAT
Retained AION
Retained NAS
Retained LRC
Retained GNT
Retained PPT
New add MCO
New add THETA
40 full! Do not add  WAX
40 full! Do not add  GXS
40 full! Do not add  IOTX
40 full! Do not add  ENG
40 full! Do not add  WAN
40 full! Do not add  ARK
40 full! Do not add  CVC
40 full! Do not add  ARDR
40 full! Do not add  LOOM
40 full! Do not add  EDO


In [122]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [123]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [124]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

Application adds ['MCO' 'THETA']
Application deletes ['DGD']
Application count members: 25

Currency adds []
Currency deletes ['HC']
Currency count members: 18

ETH adds ['MCO']
ETH deletes ['DGD']
ETH count members: 23

20 adds ['RVN']
20 deletes ['BTG']
20 count members: 20

40 adds ['BTG' 'MCO' 'THETA']
40 deletes ['DGD' 'HC' 'RVN']
40 count members: 40

10 adds []
10 deletes []
10 count members: 10

Total adds ['MCO' 'THETA']
Total deletes ['DGD' 'HC']
Total count members: 70

Platform adds []
Platform deletes []
Platform count members: 27

Sector count members: 70


In [125]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])